# Downloading the data

This code relates to the paper "Measuring Financial Time Series Similarity With a View to Identifying Profitable Stock Market Opportunities" which was published in the proceedings of the International Conference on Case Based Reasoning (ICCBR) 2021

For queries please email rian.dolphin@ucdconnect.ie

--------------------------------------

This notebook contains the code to download and clean the stock price data from Yahoo Finance and compute the returns. It is worth noting that the yfinance package is not an official API and the data downloaded will not be identical each time. Sometimes a certain ticker might be unavailable etc. 

At the end of this notebook we will have constructed and saved the 'train_df.json' file which contains all the cases for use in following notebooks.

### Imports

In [14]:
#-- Imports
import yfinance as yf
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
#from scipy.special import softmax
import time

import datetime as dt
import random

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

#-- Uncomment these for white Theme
my_template = 'plotly_white'
background_color = 'rgba(255,255,255,1)'

#-- Dark Theme
#my_template = 'plotly_dark'
#background_color = 'rgba(0,0,0,1)'

#-- Dark Transparent background
#my_template = 'plotly_dark'
#background_color = 'rgba(0,0,0,0)'

### Load Data
As described in the paper, the selection criterion for stocks was their inclusion in one of the major indices listed. To get these constituents the holdings csv file from the iShares ETFs were used.

In [15]:
#-- Load in different iShares data
df_nasdaq = pd.read_csv('Raw_Data/nasdaq_100_ishares_raw.csv', skiprows=2)
df_eurostoxx = pd.read_csv('Raw_Data/eurostoxx_ishares_raw.csv', skiprows=2)
df_FTSE = pd.read_csv('Raw_Data/FTSE_ishares_raw.csv', skiprows=2)
df_DAX = pd.read_csv('Raw_Data/DAX_ishares_raw.csv', skiprows=2)

In [16]:
#-- Remove unnecessary info and add Yahoo Finance exchange code
def clean_ishares_type1(df):
    df = df.drop([
     'Weight (%)',
     'Price',
     'Nominal',
     'Market Value',
     'Notional Value'],
                axis=1)
    #-- Remove cash & derivatives
    df = df[df['Asset Class']=='Equity']
    df.rename(columns={'Issuer Ticker':'Ticker', 'Market Currency':'Currency'}, inplace=True)
    df.reset_index(inplace=True, drop=True)
    
    #-- Define map from iShares name to Yahoo Finance name
    exchange_map_yf = {'Nyse Euronext - Euronext Paris':'PA',
                       'Xetra':'DE',
                       'Euronext Amsterdam':'AS',
                       'Bolsa De Madrid':'MC',
                       'Borsa Italiana':'MI',
                       'Nasdaq Omx Helsinki Ltd.':'HE',
                       'Irish Stock Exchange - All Market':'IR',
                       'Nyse Euronext - Euronext Brussels':'BR',
                       'NASDAQ':'',
                       'London Stock Exchange':'L'}

    df['yf_exchange_code'] = None
    for row in range(len(df)):
        df.yf_exchange_code[row] = exchange_map_yf[df.Exchange[row]]
        
    #-- Make the index the Yahoo Finance adapted ticker
    # Easier for indexing later
    df.index = df.Ticker + "." + df.yf_exchange_code
    df.index = df.index.str.rstrip('.')
    
    return df
#df = clean_ishares_type1(df_DAX)
#df.head()

#-- Download price data from cleaned up dataframe
def download_from_df(df, start, end):
    """
    Input: Dataframe output from clean_ishares_type1()
    Output: Dataframe will all of the price data
    """
    #-- Get the ticker string to feed into yf.download()
    # Format: space seperated YF adapted tickers
    ticker_string = ""
    for row in range(len(df)):
        ticker_string = ticker_string + df.index[row] + " "
            
    #-- Download the data
    return yf.download(ticker_string, start=start, end=end, group_by='ticker')

#-- Download prices from a list of uncleaned iShares dataframes
def multiple_download_list(df_list_uncleaned, start, end):
    """
    Input:
        - List of iShares original csv files
        - start, end dates
    """
    data_list = []
    for i in range(0,len(df_list_uncleaned)):
        print("="*20)
        print(f'Download {i+1} of {len(df_list_uncleaned)}')
        temp = download_from_df(clean_ishares_type1(df_list_uncleaned[i]), start=start, end=end)
        data_list.append(temp)
        
    del temp
    return data_list

data_list = multiple_download_list([df_nasdaq, df_eurostoxx, df_FTSE],start="2005-01-01", end="2021-01-01")

In [17]:
#data_list = multiple_download_list([df_DAX, df_FTSE, df_eurostoxx, df_nasdaq],start="2015-01-01", end="2020-01-01")
data_list = multiple_download_list([df_nasdaq, df_eurostoxx, df_FTSE],start="2005-01-01", end="2021-01-01")

Download 1 of 3
[*********************100%***********************]  103 of 103 completed
Download 2 of 3
[*********************100%***********************]  50 of 50 completed

1 Failed download:
- UNA.AS: No data found for this date range, symbol may be delisted
Download 3 of 3
[*********************100%***********************]  102 of 102 completed

4 Failed downloads:
- LSE.L: No data found, symbol may be delisted
- RRN.L: No data found, symbol may be delisted
- GVC.L: No data found, symbol may be delisted
- BT.A.L: No data found, symbol may be delisted


### Clean Data

In [18]:
#-- Drop columns with over X% NaN
    #- Forward and back fill remaining NaNs
for i in range(len(data_list)):
    print(data_list[i].isnull().sum().sum())
    #data.dropna(axis=1, thresh=len(data)*0.8, inplace=True)
    data_list[i]=data_list[i].groupby(axis=1, level=0).filter(lambda d: ~(d.isna().sum().sum()>np.product(d.shape)*0.01))
    data_list[i].columns = data_list[i].columns.remove_unused_levels()
    #-- Fill the days which are holidays in some markets but not others by back/forward fill
    data_list[i]=data_list[i].fillna(method='ffill')
    data_list[i]=data_list[i].fillna(method='bfill')
    print(data_list[i].isnull().sum().sum())

282078
0
117468
0
324588
0


In [19]:
#-- Get Returns Data
returns_list = []
for data in data_list:
    returns_list.append(data.pct_change().iloc[1:])
    
#-- Remove NaN from Returns Data
for i in range(len(returns_list)):
    """
    NaN values in returns data usually stems from recurring 0's in volume column of price data
    """
    #-- Infinity volume pct change if volume 0 - set to nan so can be removed if a lot
    returns_list[i] = returns_list[i].replace([np.inf, -np.inf], np.nan)
    print('='*20)
    print(f'Dataset {i}')
    #-- Drop column with over X% NaN
    print(f'NaN Before: {returns_list[i].isnull().sum().sum()}')
    #data.dropna(axis=1, thresh=len(data)*0.8, inplace=True)
    returns_list[i]=returns_list[i].groupby(axis=1, level=0).filter(lambda d: ~(d.isna().sum().sum()>np.product(d.shape)*0.01))
    returns_list[i].columns = returns_list[i].columns.remove_unused_levels()
    #-- Fill the days which are holidays in some markets but not others by back/forward fill
    if returns_list[i].isnull().sum().sum() > 10:
        print(f'Warning - Investigate further - Still {returns_list[i].isnull().sum().sum()} NaN Values')
    returns_list[i]=returns_list[i].fillna(method='ffill')
    returns_list[i]=returns_list[i].fillna(method='bfill')
    print(f'NaN After: {returns_list[i].isnull().sum().sum()}')

Dataset 0
NaN Before: 8
NaN After: 0
Dataset 1
NaN Before: 133
Warning - Investigate further - Still 133 NaN Values
NaN After: 0
Dataset 2
NaN Before: 3381
Warning - Investigate further - Still 1302 NaN Values
NaN After: 0


### Drop certain stocks due to incorrect data

Some months returns are below -1 which is obviously incorrect.

In [20]:
print(len(returns_list[2].columns))

420


In [21]:
for i in range(len(returns_list)):
    #-- Drop tesco due to incorrect data
    if 'TSCO.L' in returns_list[i].columns.get_level_values(0):
        returns_list[i] = returns_list[i].drop('TSCO.L', axis=1, level=0)

In [22]:
for i in range(len(returns_list)):
    #-- Drop tickers for different share classes
    #- I.e. we don't want GOOG and GOOGL
    for temp_ticker in ['RDSB.L', 'GOOGL', 'LBTYK']:
        if temp_ticker in returns_list[i].columns.get_level_values(0):
            returns_list[i] = returns_list[i].drop(temp_ticker, axis=1, level=0)

In [23]:
print(len(returns_list[2].columns))

408


### Change to monthly returns

In [24]:
if True:
    for i in range(len(returns_list)):
        returns_list[i].index = pd.to_datetime(returns_list[i].index)
        returns_list[i] = returns_list[i].resample('M').apply(lambda x: ((x + 1).cumprod() - 1).last("D"))
    print("===== Completed =====")

===== Completed =====


### Change to Weekly

In [12]:
#-- NOT USED
if False:
    for i in range(len(returns_list)):
        returns_list[i].index = pd.to_datetime(returns_list[i].index)
        returns_list[i] = returns_list[i][::5]

### Excess Return

In [9]:
#-- NOT USED
if False:
    #-- Change Adj Close to Excess Return of Adjusted Close
    for df_num in range(len(returns_list)):
        for date in returns_list[df_num].index:
            temp = returns_list[df_num].loc[date]
            temp[temp.index.get_level_values(1)=='Adj Close'] = temp[temp.index.get_level_values(1)=='Adj Close'] - temp[temp.index.get_level_values(1)=='Adj Close'].mean()


### Get the train, validation & test DataFrames

In [25]:
def get_tvt_dfs(returns_list):
    #-- get all n month periods and their next month return
    #-- Propotion of train, test, val split
    train_pct = 1
    val_pct = 0
    test_pct = 0
    
    seq_len = 12 #-- MONTHLY - 12 originally
    
    
    final_columns = ['start_date', 'end_date', 'ticker','historic_returns','next_month','next_year']
    final_train_df = []
    final_val_df = []
    final_test_df = []
    
    #-- Store the raw returns data of the training, val, test periods
    train_df_list = []
    val_df_list = []
    test_df_list = []

    for i in range(len(returns_list)):
        train_index = int(len(returns_list[i])*train_pct)
        val_index = int(len(returns_list[i])*(train_pct+val_pct))

        train_df = returns_list[i].iloc[:train_index]
        train_df_list.append(train_df)

        for df, final_df in [[train_df, final_train_df]]:
            print(f'Raw Data {i+1} of {len(returns_list)} : Next TVT')
            for ticker in list(df.columns.get_level_values(0).unique()):
                temp_data = df[ticker].drop(['Open', 'High','Low','Close','Volume'], axis=1)

                for j in range(len(temp_data)-seq_len):
                    #-- the hard coded 3 in this line tells us we are predicting adjusted closing
                    start_date = temp_data.index[j]
                    end_date = temp_data.index[j+seq_len]
                    n_day_returns = temp_data.iloc[j:j+seq_len].values.flatten()
                    next_year_return = (temp_data.iloc[j+seq_len:j+seq_len+13].values+1).cumprod()[-1]-1 ##-- 13 MONTHLY, 22 DAILY
                    next_month_return = temp_data.iloc[j+seq_len].values[0]
                    #pairs.append(([ticker, temp_data.iloc[j:j+seq_len-1].values], target))
                    #-- ['start_date', 'end_date', 'ticker','historic_returns','next_month', 'next_year']
                    #final_df.loc[len(final_df)] = [start_date, end_date, ticker, n_day_returns, next_month_return, next_year_return]
                    final_df.append([start_date, end_date, ticker, n_day_returns, next_month_return, next_year_return])
                    
    final_train_df = pd.DataFrame(final_train_df, columns=final_columns)
    
    return final_train_df
                    
                    

train_df = get_tvt_dfs(returns_list)

Raw Data 1 of 3 : Next TVT
Raw Data 2 of 3 : Next TVT
Raw Data 3 of 3 : Next TVT


In [34]:
#-- Check for outliers
train_df[train_df.next_month>3]
#-- Drop outliers if present
#train_df = train_df.drop(train_df[train_df.next_month>3].index, axis=0)

,start_date,end_date,ticker,historic_returns,next_month,next_year


In [ ]:
#-- Check for outliers
train_df[train_df.next_month<-0.7]

In [35]:
#-- Save the resulting dataframe
#train_df.to_json('train_df_2021.json')

In [36]:
train_df

,start_date,end_date,ticker,historic_returns,next_month,next_year
0,2005-01-31,2006-01-31,AMD,"[-0.26202707597661856, 0.10443041470165015, -0...",0.366013,-0.491830
1,2005-02-28,2006-02-28,AMD,"[0.10443041470165015, -0.07621775733848246, -0...",-0.074880,-0.639474
2,2005-03-31,2006-03-31,AMD,"[-0.07621775733848246, -0.11724573192310983, 0...",-0.142488,-0.662270
3,2005-04-30,2006-04-30,AMD,"[-0.11724573192310983, 0.15249473971191163, 0....",-0.024427,-0.583233
4,2005-05-31,2006-05-31,AMD,"[0.15249473971191163, 0.057317107068436846, 0....",-0.045131,-0.558887
...,...,...,...,...,...,...
29875,2019-08-31,2020-08-31,SPX.L,"[-0.10783760220172289, -0.022429877069416015, ...",-0.006305,0.098849
29876,2019-09-30,2020-09-30,SPX.L,"[-0.022429877069416015, 0.01446899046617589, 0...",0.080039,0.105820
29877,2019-10-31,2020-10-31,SPX.L,"[0.01446899046617589, 0.12618294563048482, -0....",0.022511,0.023871
29878,2019-11-30,2020-11-30,SPX.L,"[0.12618294563048482, -0.003921518255960099, 0...",-0.012855,0.001330
